# Generate Finetune Data using LLMs

It is expensive to get data from humans. We can use an existing model to generate data for finetuning. Woohoo, laziness!

Here is an example with courses scraped from the UH Hilo website.

In [2]:
import json
import ollama
from tqdm.notebook import tqdm

In [12]:
# TODO: replace this with your own data
courses = json.load(open(r"C:\Users\peyto\Desktop\school24\497\labs\courses.json", encoding='UTF-8'))

In [13]:
def generate_question(description):
    response = ollama.chat(
        model="llama3.2:1b",
        messages=[
            {
                "role": "user",
                "content": f"Given the following course description, generate a question that a student would ask about this course. The student's question should mention the name of the course or the course number.\nCourse Description: {description}\nOnly reply with the student's question and nothing else."
            }
        ]
    )
    return response

def generate_answer(description, question):
    response = ollama.chat(
        model="llama3.2:1b",
        messages=[
            {
                "role": "user",
                "content": f"Given the following course description, answer the question.\nCourse Description: {description}\nQuestion: {question}\nJust reply with the answer."
            }
        ]
    )
    return response

In [14]:
description = "JPST 201 Intermediate Japanese I (4) (lecture/lab) Continuation of JPNS 102. More advanced colloquial structures and kanji. Pre: JPNS 102 or equivalent. (Same as JPNS 201) (Attributes: DH, GAHP, GH, GL)"
r = generate_question(description)

In [16]:
qa_pairs = []
for course in tqdm(courses):
    resp = generate_question(course["description"])
    question = resp["message"]["content"]

    resp = generate_answer(course["description"], question)
    answer = resp["message"]["content"]
    qa_pairs.append((question, answer))

  0%|          | 0/2312 [00:00<?, ?it/s]

In [17]:
with open("qa_pairs.tsv", "w") as fout:
    for q, a in qa_pairs:
        j = [{'role': 'user', 'content': q}, {'role': 'assistant', 'content': a} ]
        print(json.dumps(j), sep='\t', file=fout)

In [18]:
answer = generate_answer(description, question)["message"]["content"]

In [19]:
answer

'The directed studies in ENGR 99 are typically 1-hour lectures and 2 hours of lab work per week, for a total of 3 academic units (or credit).'

In [20]:
qa_pairs[-10]

('What is the assignment prompt for ENGR 103 Engineering the Future II?',
 'There is no assignment prompt provided in the course description.')